# Healthcare Payer-Provider Claims Reconciliation

This script demonstrates multi-level matching of patient encounters between payer and provider datasets, and generates a high-level summary of payment and underpayment patterns.

**Author**: Ryan Williamson  
**Date**: 2025-07

**Disclaimer**: All data and files in this repository are synthetic and for demonstration purposes only. No real patient, provider, or payer data is included.
All code is original and does not contain any proprietary logic, data, or assets.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# ---- 1. Data Load & Preparation ----
payer_data = pd.read_csv('../data/payer_data_clean.csv')
provider_claims = pd.read_csv('../data/provider_claims.csv')

all_claims_unique_ct = payer_data['claim_number'].nunique()
payor_data_staging = payer_data.copy()
matched_frames = []

In [3]:
def print_level_stats(level_name, match_df, already_mapped, total):
    unique_matched = match_df['claim_number'].nunique()
    running_total = already_mapped + unique_matched
    pct = (running_total / total) * 100
    print(f"\n{level_name} Results:")
    print(f"  - Unique claims matched this level:  {unique_matched}")
    print(f"  - Running total matched:             {running_total}")
    print(f"  - % mapped so far:                   {pct:.2f}%")
    print(f"  - Remaining:                         {total - running_total}")

running_total = 0

In [4]:
# ---- 2. Stepwise Matching & Reconciliation ----
#
# Matching proceeds in order: claim number, account number, demographics, and then less strict/fuzzy matches.
# Once matched, a claim is not considered in further levels.

In [5]:
# Level 1 - Exact match on claim number (payer control number)  
match_keys = ['claim_number']
l1 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l1['match_source'] = 'l1'
matched_frames.append(l1)
print_level_stats("Level 1 (Claim Number)", l1, running_total, all_claims_unique_ct)
running_total += l1['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l1['claim_number'])]


Level 1 (Claim Number) Results:
  - Unique claims matched this level:  295
  - Running total matched:             295
  - % mapped so far:                   59.00%
  - Remaining:                         205


In [6]:
# Level 2 - Match on account number or patient control number 
match_keys = ['account_number']
l2 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l2['match_source'] = 'l2'
matched_frames.append(l2)
print_level_stats("Level 2 (Account Number)", l2, running_total, all_claims_unique_ct)
running_total += l2['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l2['claim_number'])]


Level 2 (Account Number) Results:
  - Unique claims matched this level:  104
  - Running total matched:             399
  - % mapped so far:                   79.80%
  - Remaining:                         101


In [7]:
# Level 3 - Demographic: policy ID, DOB, service date, name fragments
match_keys = ['policy_id', 'dob', 'service_date', 'f4', 'l5']
l3 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l3['match_source'] = 'l3'
matched_frames.append(l3)
print_level_stats("Level 3 (Demographics)", l3, running_total, all_claims_unique_ct)
running_total += l3['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l3['claim_number'])]


Level 3 (Demographics) Results:
  - Unique claims matched this level:  15
  - Running total matched:             414
  - % mapped so far:                   82.80%
  - Remaining:                         86


In [8]:
# Level 3.1 - Demographics + procedure code (CPT)        
match_keys = ['dob', 'service_date', 'f4', 'l5', 'proc_code']
l3_1 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l3_1['match_source'] = 'l3_1'
matched_frames.append(l3_1)
print_level_stats("Level 3.1 (Demographics + CPT)", l3_1, running_total, all_claims_unique_ct)
running_total += l3_1['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l3_1['claim_number'])]


Level 3.1 (Demographics + CPT) Results:
  - Unique claims matched this level:  9
  - Running total matched:             423
  - % mapped so far:                   84.60%
  - Remaining:                         77


In [9]:
# Level 3.2 - Fuzzy initials + DOB, service date, procedure code 
match_keys = ['f1', 'l2', 'dob', 'service_date', 'proc_code']
l3_2 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']],left_on=match_keys, right_on=match_keys, how='inner')
l3_2['match_source'] = 'l3_2'
matched_frames.append(l3_2)
print_level_stats("Level 3.2 (Initials, Demographics + CPT)", l3_2, running_total, all_claims_unique_ct)
running_total += l3_2['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l3_2['claim_number'])]


Level 3.2 (Initials, Demographics + CPT) Results:
  - Unique claims matched this level:  11
  - Running total matched:             434
  - % mapped so far:                   86.80%
  - Remaining:                         66


In [10]:
# Level 3.3 - Name fragments + service date + procedure code 
match_keys = ['f4', 'l5', 'service_date', 'proc_code']
l3_3 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l3_3['match_source'] = 'l3_3'
matched_frames.append(l3_3)
print_level_stats("Level 3.3 (Name, Service Date + CPT)", l3_3, running_total, all_claims_unique_ct)
running_total += l3_3['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l3_3['claim_number'])]


Level 3.3 (Name, Service Date + CPT) Results:
  - Unique claims matched this level:  12
  - Running total matched:             446
  - % mapped so far:                   89.20%
  - Remaining:                         54


In [11]:
#Level 3.4 - Initials + service date + procedure code 
match_keys = ['f1', 'l2', 'service_date', 'proc_code']
l3_4 = payor_data_staging.merge(provider_claims[match_keys + ['mrn']], left_on=match_keys, right_on=match_keys, how='inner')
l3_4['match_source'] = 'l3_4'
matched_frames.append(l3_4)
print_level_stats("Level 3.4 (Initials, Service Date + CPT)", l3_4, running_total, all_claims_unique_ct)
running_total += l3_4['claim_number'].nunique()
payor_data_staging = payor_data_staging[~payor_data_staging['claim_number'].isin(l3_4['claim_number'])]


Level 3.4 (Initials, Service Date + CPT) Results:
  - Unique claims matched this level:  11
  - Running total matched:             457
  - % mapped so far:                   91.40%
  - Remaining:                         43


In [12]:
# -- Combine All Matches --
payor_data_matched = pd.concat(matched_frames, ignore_index=True)

### MRN/Encounter Crosswalk Logic: Assigned vs Mapped MRN

In the reconciliation process, it is common for a single provider MRN to match multiple payer encounter numbers  
(i.e., the same MRN can be **mapped** to more than one payer `claim_number`). This occurs in real-world cases, such as duplicate submissions, corrections, or payer-side processing, where a single patient encounter may result in multiple payer claims.

It is also possible, though less common, for a single payer claim to match to more than one MRN, especially with imperfect or overlapping source data.  
**For simplicity and reproducibility, this code selects only the first MRN for each claim if duplicates exist,** ensuring each claim appears only once in the crosswalk and preventing ambiguity in downstream reporting.

We distinguish between:

- **Assigned MRN:**  
  For each MRN, the *first* payer encounter that matches is designated as the **assigned** MRN.  
  This ensures a unique (1:1) link between MRN and claim_number.
  
- **Mapped MRN:**  
  All payer encounters that match a given MRN (including the assigned one) are recorded in the `mapped_mrn` column.  
  This captures the full one-to-many relationship, making it clear when an MRN is linked to multiple claims.

If a payer claim cannot be matched to any provider MRN, both columns are null and the `match_status` is set to **"Unmapped"**.

> **Note:**  
> If you need to preserve all possible many-to-many relationships (i.e., all (claim_number, MRN) pairs),  
> you can adjust the logic to keep all pairs before deduplication.


In [13]:
# ---- 3. MRN/Encounter Assignment & Crosswalk Construction ----
#
# Build assigned vs mapped MRN logic for auditability.
# See the markdown cell above for explanation of assigned/mapped MRN.

In [14]:
pairs = payor_data_matched[['claim_number', 'match_source', 'mrn']]
pairs = pairs.sort_values(['match_source', 'claim_number', 'mrn'])

mapped_df = pairs.drop_duplicates(subset=['claim_number'], keep='first').rename(columns={'mrn': 'mapped_mrn'})
assigned_df = pairs.drop_duplicates(subset=['mrn'], keep='first').rename(columns={'mrn': 'assigned_mrn'})

crosswalk = pd.merge(mapped_df, assigned_df[['claim_number', 'assigned_mrn']], on='claim_number', how='left')

In [15]:
# Example: See all claims linked to MRN8960726 to illustrate assigned vs mapped logic.

# The first claim will show assigned_mrn == mapped_mrn.
# Other claims will show assigned_mrn as NaN and mapped_mrn == MRN5989.

crosswalk[crosswalk['mapped_mrn'] == 'MRN8960726']

,claim_number,match_source,mapped_mrn,assigned_mrn
67,CLM2612ET,l1,MRN8960726,MRN8960726
387,CLM8616WD,l2,MRN8960726,NaN


In [16]:
# ---- 4. Merge Crosswalk to Full Claims and Assign Match Status ----
#
# Produces a final, auditable file with one row per payer claim, showing assignment, mapping, and match status.

In [17]:
payer_data_out = payer_data[['claim_number']].merge(
    crosswalk,
    on='claim_number',
    how='left'
)

payer_data_out['match_status'] = np.where(
    payer_data_out['mapped_mrn'].isna(),
    'Unmapped',
    np.where(
        payer_data_out['assigned_mrn'].notna(),
        'Provider MRN Mapped',
        'Provider MRN Mapped, but same MRN exists for a different claim.'
    )
)

payer_data_out['match_source'] = payer_data_out['match_source'].fillna('Unmapped')

In [18]:
# ---- 5. Reconciliation Reporting & Mapping Export ----
#
# Shows overall reconciliation stats, level-by-level mapping, and breakdown by match status for audit and business reporting.

In [19]:
total_claims = len(payer_data_out)
num_found = (payer_data_out['match_status'] != 'Unmapped').sum()
pct_found = num_found / total_claims * 100
num_unique_assigned = payer_data_out['assigned_mrn'].notna().sum()
num_duplicate_mapped = ((payer_data_out['assigned_mrn'].isna()) & (payer_data_out['mapped_mrn'].notna())).sum()
num_unmapped = (payer_data_out['mapped_mrn'].isna()).sum()

# Level-by-level summary
level_summary = (
    payer_data_out[payer_data_out['mapped_mrn'].notna()]
        .groupby('match_source')['claim_number']
        .nunique()
        .reset_index()
        .rename(columns={'claim_number': 'num_claims_mapped'})
        .sort_values('num_claims_mapped', ascending=False)
)

print("="*60)
print("Payer-Provider Reconciliation Summary")
print("="*60)
print('')

print(f"Total claims analyzed:          {total_claims:,}")
print(f"Total matched to provider MRN:  {num_found:,} ({pct_found:.1f}%)")
print(f"  - Uniquely assigned MRNs:     {num_unique_assigned:,}")
print(f"  - Duplicate mapped MRNs:      {num_duplicate_mapped:,}")
print(f"Total unmapped claims:          {num_unmapped:,} ({num_unmapped/total_claims*100:.1f}%)\n")

level_summary = (
    payer_data_out[payer_data_out['mapped_mrn'].notna()]
        .groupby('match_source')['claim_number']
        .nunique()
        .reset_index()
        .rename(columns={'claim_number': 'num_claims_mapped'})
        .sort_values('match_source')
)

print("Claims mapped by level:")
print(level_summary)

print("\nCLaims by match status:")
print(payer_data_out['match_status'].value_counts())
print("="*60)

Payer-Provider Reconciliation Summary

Total claims analyzed:          500
Total matched to provider MRN:  457 (91.4%)
  - Uniquely assigned MRNs:     456
  - Duplicate mapped MRNs:      1
Total unmapped claims:          43 (8.6%)

Claims mapped by level:
  match_source  num_claims_mapped
0           l1                295
1           l2                104
2           l3                 15
3         l3_1                  9
4         l3_2                 11
5         l3_3                 12
6         l3_4                 11

CLaims by match status:
Provider MRN Mapped                                                456
Unmapped                                                            43
Provider MRN Mapped, but same MRN exists for a different claim.      1
Name: match_status, dtype: int64


### Additional Reconciliation Notes: Provider-Side Gaps

In addition to unmapped payer claims, the workflow also identified provider claims that did not match to any payer record.

These usually reflect:

- **Bad data** (clearinghouse provided incorrect payer information to the provider)
- **Unbilled or held claims** (claims never submitted, or held for additional information)
- **Claims rejected or denied** by the payer and not included in TIN-level reports
- **Timing differences** (claims still in process or pending adjudication)

Identifying these unmatched MRNs allows revenue cycle teams to:

- Cross-check with clearinghouse submissions  
- Investigate why services were not paid or submitted  
- Flag breakdowns in claim handoffs

These unmatched provider claims are summarized and exported separately to support revenue cycle management workflows.

In [20]:
unmapped_provider_claims = provider_claims[~provider_claims['mrn'].isin(payer_data_out['mapped_mrn'])]
unmapped_provider_claims.to_csv('../recon_results/unmapped_provider_claims.csv', index = False)
print(f"Total unmatched provider records: {len(unmapped_provider_claims)}")
print("Sample of unmapped_provider_claims:")
print(unmapped_provider_claims[['mrn', 'charge_amt', 'contracted_amt', 'insurance_paid_amt']].sample(5))

Total unmatched provider records: 294
Sample of unmapped_provider_claims:
            mrn  charge_amt  contracted_amt  insurance_paid_amt
550  MRN9377323          80              20                 0.0
517  MRN7435275         125              20                 0.0
688  MRN8236708         125              20                 0.0
485  MRN1800720          75              10                 0.0
507  MRN9270404         300             130                 0.0


In [21]:
# Export - Typically, I store data as a parquet. But, for this example, I am using csv.
payer_data_out.to_csv('../recon_results/claims_crosswalk.csv', index=False)
# payer_data_out.to_parquet('../recon_results/final_claims_crosswalk.parquet.gzip', compression='gzip')

print("Sample of payer_data_out:")
print(payer_data_out.sample(10))

Sample of payer_data_out:
    claim_number match_source  mapped_mrn assigned_mrn         match_status
494    CLM4078TS     Unmapped         NaN          NaN             Unmapped
345    CLM4082TD           l1  MRN9839546   MRN9839546  Provider MRN Mapped
19     CLM9447XH         l3_1  MRN7246306   MRN7246306  Provider MRN Mapped
302    CLM3067QJ           l1  MRN2618580   MRN2618580  Provider MRN Mapped
379    CLM0385PT           l2  MRN1022078   MRN1022078  Provider MRN Mapped
463    CLM7798UL     Unmapped         NaN          NaN             Unmapped
199    CLM5221HF           l1  MRN2746448   MRN2746448  Provider MRN Mapped
422    CLM7785LA           l2  MRN1904819   MRN1904819  Provider MRN Mapped
447    CLM4381QG           l2  MRN1406220   MRN1406220  Provider MRN Mapped
364    CLM3517LG           l2  MRN6954462   MRN6954462  Provider MRN Mapped


In [22]:
# ---- 6. Payment Performance and Underpayment Patterns ----
#
# Analyzes how actual payments compare to contracted rates for all procedures, quantifies underpayment trends, and identifies codes with the largest revenue gaps.

In [23]:
merged = pd.merge(payer_data_out, payer_data[['claim_number', 'proc_code', 'paid_amt']], on = 'claim_number', how = 'left')
merged = pd.merge(merged, provider_claims[['mrn', 'charge_amt', 'contracted_amt', 'insurance_paid_amt']], left_on = 'assigned_mrn', right_on = 'mrn', how = 'left')
merged = merged.drop(columns = ['mrn'])

merged.to_csv('../recon_results/payer_provider_data_merged.csv', index=False)
merged.head(5)

,claim_number,match_source,mapped_mrn,assigned_mrn,match_status,proc_code,paid_amt,charge_amt,contracted_amt,insurance_paid_amt
0,CLM8793BJ,l3,MRN9265965,MRN9265965,Provider MRN Mapped,87086,15.0,100.0,15.0,15.0
1,CLM0985CX,l3,MRN2060428,MRN2060428,Provider MRN Mapped,80050,2.0,80.0,20.0,2.0
2,CLM3057BZ,l3,MRN0092626,MRN0092626,Provider MRN Mapped,81001,20.0,80.0,20.0,20.0
3,CLM7609DX,l3,MRN9840345,MRN9840345,Provider MRN Mapped,71020,100.0,250.0,100.0,100.0
4,CLM1678MG,l3,MRN4073490,MRN4073490,Provider MRN Mapped,81001,13.0,300.0,130.0,13.0


In [24]:
# Underpayment flags and ratios
merged['underpaid'] = merged['paid_amt'] < merged['contracted_amt']
merged['underpaid_amt'] = merged['contracted_amt'] - merged['paid_amt']
merged['pct_underpaid'] = merged['underpaid_amt'] / merged['contracted_amt']

summary = (
    merged.groupby('proc_code')
    .agg(
        total_claims = ('proc_code', 'size'),
        underpaid_claims = ('underpaid', 'sum'),
        pct_underpaid_claims = ('underpaid', lambda x: 100 * x.sum() / len(x)),
        total_contracted_amt = ('contracted_amt', 'sum'),
        total_paid_amt = ('paid_amt', 'sum'),
        total_underpaid_amt = ('underpaid_amt', lambda x: x[merged.loc[x.index, 'underpaid']].sum()),
        avg_underpaid_amt = ('underpaid_amt', lambda x: x[merged.loc[x.index, 'underpaid']].mean()),
        mean_pct_underpaid = ('pct_underpaid', lambda x: x[merged.loc[x.index, 'underpaid']].mean()),
        payment_ratio = ('paid_amt', lambda x: 100 * x.sum() / merged.loc[x.index, 'contracted_amt'].sum())
    )
    .sort_values('total_underpaid_amt', ascending=False)
    .reset_index()
)

# Summary output by CPT (proc_code)
summary = summary.fillna(0).round(2)
summary.to_csv('../recon_results/underpayment_summary.csv', index=False)
summary.head(50)

,proc_code,total_claims,underpaid_claims,pct_underpaid_claims,total_contracted_amt,total_paid_amt,total_underpaid_amt,avg_underpaid_amt,mean_pct_underpaid,payment_ratio
0,36415,46,7,15.22,4085.0,3485.00,600.00,85.71,0.95,85.31
1,71020,49,3,6.12,5445.0,5068.00,377.00,125.67,0.97,93.08
2,81001,60,6,10.00,2850.0,2513.00,337.00,56.17,0.88,88.18
3,93000,50,5,10.00,1490.0,1332.50,157.50,31.50,0.90,89.43
4,99203,47,2,4.26,775.0,643.00,132.00,66.00,0.95,82.97
5,99213,59,6,10.17,1180.0,1076.00,104.00,17.33,0.87,91.19
6,99214,51,3,5.88,900.0,846.00,54.00,18.00,0.90,94.00
7,80050,39,2,5.13,710.0,674.00,36.00,18.00,0.90,94.93
8,90471,53,3,5.66,480.0,466.25,13.75,4.58,0.92,97.14
9,87086,46,1,2.17,435.0,425.00,10.00,10.00,1.00,97.70
